In [7]:
import torch
import torch.nn as nn 
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [2]:
# Dataset class
class MultiTaskDataset(Dataset):
    def __init__(self, df, tfms, size=64):
        self.paths = list(df.name)
        self.labels = list(df.label)
        self.tfms = tfms
        self.size = size
        self.norm = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) #image net
        
    def __len__(self): return len(self.paths)
    
    def __getitem__(self, idx): # using FastAI’s data augmentations instead of torchvision.transform
        # dealing with the image
        img = PIL.Image.open(self.paths[idx]).convert('RGB')
        img = Image(pil2tensor(img, dtype=np.float32).div_(255))
        img = img.apply_tfms(self.tfms, size = self.size)
        img = self.norm(img.data)
        
        # dealing with the labels
        labels = self.labels[idx].split(" ")
        age = torch.tensor(float(labels[0]), dtype=torch.float32)
        gender = torch.tensor(int(labels[1]), dtype=torch.int64)
        ethnicity = torch.tensor(int(labels), dtype=torch.int64)
        
        return img.data, (age.log_()/4.75, gender, ethnicity)  # Taking logs means that errors in predicting high ages and low ages will affect the result equally
        # divide by 4.75 as max value of log(age) => results should be between 0 and 1 and MSE shouldn't be bigger than other losses.

NameError: name 'Dataset' is not defined

In [5]:
# Creating dataloader
tfms = get_transforms()  #?
train_ds = MultiTaskDataset(df_train, tfms[0], size=64)
valid_ds = MultiTaskDataset(df_valid, tfms[1], size=64)
train_dl = DataLoader(train_ds, batch_size=128, shuffle=True, num_workers = 2)
valid_dl = DataLoader(valid_ds, batch_size=128, shuffle=False, num_workers=2)
data = DataBunch(train_dl, valid_dl)   #?

NameError: name 'get_transforms' is not defined

In [8]:
# Creating the model
class MultiTaskModel(nn.Module):
    """_summary_
    Create a MTL model with the encoder from "arch" and with dropout multiplier ps.

    Args:
        nn (_type_): _description_
    """
    def __init__(self, arch, ps=0.5):   # arch ? How to input diff target for age, gender, ethnic?
        super(MultiTaskModel, self).__init__()
        self.endcoder = create_body(arch)   # FastAI function that creates an encoder given an architechture 
        self.fc1 = create_head(1024, 1, ps=ps) # FastAI function that creates a head ?
        self.fc2 = create_head(1024, 2, ps=ps)
        self.fc3 = create_head(1024, 5, ps = ps)
        
    def forward(self, x):
        x = self.endcoder(x)
        age = torch.sigmoid(self.fc1(x))  # take sigmoid to force model to always output a prediction in the acceptable range
        gender = self.fc2(x)
        ethnicity = self.fc3(x)
        
        return [age, gender, ethnicity]

In [10]:
# Defining loss functions
# Letting the model learn how to weight th task specific losses
class MultiTaskLossWrapper(nn.Module):
    def __init__(self ,task_num):
        super(MultiTaskLossWrapper, self).__init__()
        self.task_num = task_num
        self.log_vars  = nn.Parameter(torch.zeros((task_num)))
        
    def forward(self, preds, age, gender, ethnicity):
        mse, crossEntropy = MSELossFlat(), CrossEntropyFlat() #?
        
        loss0 = mse(preds[0], age)
        loss1 = crossEntropy(preds[1], gender)
        loss2 = crossEntropy(preds[2], ethnicity)
        
        precision0 = torch.exp(-self.log_vars[0])
        loss0 = precision0 + loss0 + self.log_vars[0]
        
        precision1 = torch.exp(-self.log_vars[1])
        loss1 = precision1 + loss1 + self.log_vars[1]
        
        precision2 = torch.exp(-self.log_vars[2])
        loss2 = precision2 + loss2 + self.log_vars[2]
        
        return loss0 + loss1 + loss2
    
# A principled approach to multi-task deep learning which weighs multiple loss functions by considering the homoscedastic uncertainty of each task. This allows us to simultaneously learn various quantities with different units or scales in both classification and regression settings.


In [ ]:
# Creating the learner and training
def rmse_age(preods, age, gender, ethnicity): return root_mean_square_error(preds[0], age)
def acc_gender(preds, age, gender, ethnicity): return accuracy(preds[1], gender)
def acc_ethnicity(preds, age, gender, ethnicity): return accuracy(preds[2], ethnicity)
metrics = [rmse_age, acc_gender, acc_ethnicity]

model = MultiTaskModel(models.resnet34, ps=0.25)

loss_func = MultiTaskLossWrapper(3).to(data.device) # making sure the loss in on the gpu

learn = Learner(data,model, loss_func = loss_func, callback_fns = ShowGraph, metrics= metrics)  #? learner

# splitting the model so that can use discriminative learning rates
learn.split([learn.model.encoder[:6],
             learn.model.encoder[6:],
             nn.ModuleList([learn.model.fc1, learn.model.fc2, learn.model.fc3])]);

# first train only the last layer group (the heads)
learn.freeze()


# After that unfreeze the encoder and train the whole model with discriminative learning rates for 100 epochs